In [1]:
import pandas as pd
import numpy as np
import allel
import json

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
refgene = pd.read_csv('data/refseq-20200109.txt', sep='\t')
refgene['length'] = refgene['txEnd'] - refgene['txStart']
refgene = refgene.sort_values('length')
refgene = refgene.drop_duplicates('name2', keep='last')

In [4]:
refgene.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length
43333,779,NR_003499.1,chr15,+,25513663,25513696,25513696,25513696,1,"25513663,","25513696,",0,SNORD115-47,none,none,"-1,",33
3416,1311,NR_039620.1,chr1,-,95211415,95211456,95211456,95211456,1,"95211415,","95211456,",0,MIR378G,none,none,"-1,",41
16216,589,NR_039661.1,chr5,-,535954,535997,535997,535997,1,"535954,","535997,",0,MIR4456,none,none,"-1,",43
4887,1911,NR_003938.1,chr1,-,173833312,173833355,173833355,173833355,1,"173833312,","173833355,",0,SNORD81,none,none,"-1,",43
14863,1479,NR_031737.1,chr4,+,117220880,117220924,117220924,117220924,1,"117220880,","117220924,",0,MIR1973,none,none,"-1,",44


In [5]:
gene_info = pd.read_csv('data/Homo_sapiens.gene_info.gz', sep='\t')

In [6]:
gene_info.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20191220,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200113,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20191221,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200209,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200121,-


In [7]:
refgene_info = refgene.merge(gene_info, left_on='name2', right_on='Symbol')

In [8]:
refgene_info.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,779,NR_003499.1,chr15,+,25513663,25513696,25513696,25513696,1,"25513663,","25513696,",0,SNORD115-47,none,none,"-1,",33,9606,100036567,SNORD115-47,-,HBII-52-46|HBII-52-47,HGNC:HGNC:33065,15,15q11.2,"small nucleolar RNA, C/D box 115-47",snoRNA,SNORD115-47,"small nucleolar RNA, C/D box 115-47",O,HBII-52-46 snoRNA,20191221,-
1,1311,NR_039620.1,chr1,-,95211415,95211456,95211456,95211456,1,"95211415,","95211456,",0,MIR378G,none,none,"-1,",41,9606,100616321,MIR378G,-,-,HGNC:HGNC:41785|Ensembl:ENSG00000263526|miRBas...,1,1p21.3,microRNA 378g,ncRNA,MIR378G,microRNA 378g,O,hsa-mir-378g,20191221,-
2,589,NR_039661.1,chr5,-,535954,535997,535997,535997,1,"535954,","535997,",0,MIR4456,none,none,"-1,",43,9606,100616381,MIR4456,-,-,HGNC:HGNC:41606|Ensembl:ENSG00000264233|miRBas...,5,5p15.33,microRNA 4456,ncRNA,MIR4456,microRNA 4456,O,hsa-mir-4456,20191221,-
3,1911,NR_003938.1,chr1,-,173833312,173833355,173833355,173833355,1,"173833312,","173833355,",0,SNORD81,none,none,"-1,",43,9606,26769,SNORD81,-,RNU104|U81|Z23,HGNC:HGNC:10101,1,1q25.1,"small nucleolar RNA, C/D box 81",snoRNA,SNORD81,"small nucleolar RNA, C/D box 81",O,"RNA, U104 small nucleolar",20191221,-
4,1479,NR_031737.1,chr4,+,117220880,117220924,117220924,117220924,1,"117220880,","117220924,",0,MIR1973,none,none,"-1,",44,9606,100302290,MIR1973,-,hsa-mir-1973|mir-1973,HGNC:HGNC:37061|Ensembl:ENSG00000284253|miRBas...,4,4q26,microRNA 1973,ncRNA,MIR1973,microRNA 1973,O,-,20191221,-


In [9]:
gene = refgene_info.loc[
    refgene_info['type_of_gene'] == 'protein-coding',
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [10]:
gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
2719,chr1,69090,70008,79501,OR4F5,NM_001005484.1,+
2832,chr1,367658,368597,729759,OR4F29,NM_001005221.2,+
2855,chr1,621095,622034,81399,OR4F16,NM_001005277.1,-
13665,chr1,861120,879961,148398,SAMD11,NM_152486.2,+
12356,chr1,879582,894679,26155,NOC2L,NM_015658.3,-


In [11]:
gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/gene-20200109.sorted.bed', index=False, sep='\t')

In [12]:
!bgzip -cf data/gene-20200109.sorted.bed > ../data/gene-20200109.sorted.bed.gz

In [13]:
!tabix -fp bed ../data/gene-20200109.sorted.bed.gz

In [14]:
curation_gene = pd.read_csv('data/ClinGen_gene_curation_list_GRCh37.tsv', sep='\t', dtype=str, skiprows=5)

In [15]:
curation_gene.head()

,#Gene Symbol,Gene ID,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,A4GALT,53947,22q13.2,chr22:43088121-43117307,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2014-12-11,111400,NaN
1,AAGAB,79719,15q23,chr15:67493013-67547536,3,Sufficient evidence for dosage pathogenicity,23064416,23000146,NaN,0,No evidence available,NaN,NaN,NaN,2013-02-28,148600,NaN
2,AARS,16,16q22.1,chr16:70286297-70323412,0,No evidence available,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2018-01-11,NaN,NaN
3,AARS2,57505,6p21.1,chr6:44266463-44281063,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN
4,AASS,10157,7q31.32,chr7:121713598-121784344,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN


In [16]:
hi_genes = set(
    curation_gene.loc[
        curation_gene['Haploinsufficiency Score'] == '3', '#Gene Symbol'
    ]
)

In [17]:
hi_gene = gene[gene['name2'].isin(hi_genes)].copy()

In [18]:
hi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
25636,chr1,6845383,7829766,23261,CAMTA1,NM_015215.3,+
17434,chr1,17345216,17380665,6390,SDHB,NM_003000.2,-
21801,chr1,27022521,27108601,8289,ARID1A,NM_139135.2,+
20929,chr1,27860755,27930005,27245,AHDC1,NM_001029882.3,-
17173,chr1,43391045,43424847,6513,SLC2A1,NM_006516.2,-


In [19]:
hi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-gene-20200109.sorted.bed', index=False, sep='\t')

In [20]:
!bgzip -cf data/hi-gene-20200109.sorted.bed > ../data/hi-gene-20200109.sorted.bed.gz

In [21]:
!tabix -fp bed ../data/hi-gene-20200109.sorted.bed.gz

In [22]:
hi_cds = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'cdsStart', 'cdsEnd', 'GeneID', 'name2', 'name']
].copy()
hi_cds = hi_cds.sort_values(['chrom', 'cdsStart', 'cdsEnd'])

In [23]:
hi_cds.head()

,chrom,cdsStart,cdsEnd,GeneID,name2,name
25636,chr1,6845590,7826551,23261,CAMTA1,NM_015215.3
17434,chr1,17345375,17380514,6390,SDHB,NM_003000.2
21801,chr1,27022894,27107247,8289,ARID1A,NM_139135.2
20929,chr1,27873814,27878626,27245,AHDC1,NM_001029882.3
17173,chr1,43392711,43424322,6513,SLC2A1,NM_006516.2


In [24]:
hi_cds.rename(columns={
    'chrom': '#chrom', 'cdsStart': 'start', 'cdsEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-cds-20200109.sorted.bed', index=False, sep='\t')

In [25]:
!bgzip -cf data/hi-cds-20200109.sorted.bed > ../data/hi-cds-20200109.sorted.bed.gz

In [26]:
!tabix -fp bed ../data/hi-cds-20200109.sorted.bed.gz

In [27]:
hi_exons = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'exonStarts', 'exonEnds', 'GeneID', 'name2', 'name', 'strand']
].copy()

In [28]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2693,chrY,"2654895,","2655782,",6736,SRY,NM_003140.2,-
3929,chr16,"86600856,","86602539,",2303,FOXC2,NM_005251.2,+
4495,chr7,"19155090,19156293,","19155754,19157295,",7291,TWIST1,NM_000474.3,-
4610,chr12,"56435685,56436208,56437146,56437902,","56435953,56436386,56437277,56438007,",6231,RPS26,NM_001029.3,+
4822,chr3,"181429711,","181432224,",6657,SOX2,NM_003106.3,+


In [29]:
hi_exons['exonStarts'] = hi_exons['exonStarts'].str.replace(r',$', '')
hi_exons['exonEnds'] = hi_exons['exonEnds'].str.replace(r',$', '')

In [30]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2693,chrY,2654895,2655782,6736,SRY,NM_003140.2,-
3929,chr16,86600856,86602539,2303,FOXC2,NM_005251.2,+
4495,chr7,"19155090,19156293","19155754,19157295",7291,TWIST1,NM_000474.3,-
4610,chr12,"56435685,56436208,56437146,56437902","56435953,56436386,56437277,56438007",6231,RPS26,NM_001029.3,+
4822,chr3,181429711,181432224,6657,SOX2,NM_003106.3,+


In [31]:
start = hi_exons['exonStarts'].str.split(',').apply(pd.Series).stack().reset_index()
start = start.rename(columns={'level_0': 'row', 0: 'start'})[['row', 'start']]
start['start'] = start['start'].astype(int)
end = hi_exons['exonEnds'].str.split(',').apply(pd.Series).stack().reset_index()
end = end.rename(columns={0: 'end'})['end'].astype(int)
position = start.join(end)

In [32]:
position.head()

,row,start,end
0,2693,2654895,2655782
1,3929,86600856,86602539
2,4495,19155090,19155754
3,4495,19156293,19157295
4,4610,56435685,56435953


In [33]:
exon = position.merge(
    hi_exons[['chrom', 'GeneID', 'name2', 'name', 'strand']], how='left', left_on='row', right_index=True
)
exon = exon.sort_values(['chrom', 'start', 'end'])

In [34]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand
4527,25636,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+
4528,25636,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+
4529,25636,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+
4530,25636,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+
4531,25636,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+


In [35]:
exon['+'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=True).astype(int)
exon['-'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=False).astype(int)

In [36]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
4527,25636,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23
4528,25636,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22
4529,25636,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21
4530,25636,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20
4531,25636,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19


In [37]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
3204,24186,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2
3205,24186,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1
63,7924,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2
64,7924,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1
0,2693,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1


In [38]:
exon['exon'] = pd.concat([exon.loc[exon['strand'] == '+', '+'], exon.loc[exon['strand'] == '-', '-']])

In [39]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
4527,25636,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1
4528,25636,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2
4529,25636,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3
4530,25636,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4
4531,25636,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5


In [40]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
3204,24186,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2,2
3205,24186,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1,1
63,7924,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2,2
64,7924,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1,1
0,2693,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1


In [41]:
exon['last_exon'] = exon.groupby(['name2', 'name'])['exon'].transform('max') == exon['exon']

In [42]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
4527,25636,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1,False
4528,25636,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2,False
4529,25636,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3,False
4530,25636,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4,False
4531,25636,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5,False


In [43]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
3204,24186,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2,2,False
3205,24186,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1,1,False
63,7924,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2,2,True
64,7924,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1,1,False
0,2693,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1,True


In [44]:
exon = exon[
    ['chrom', 'start', 'end', 'GeneID', 'name2', 'name', 'exon', 'last_exon']
].sort_values(['chrom', 'start', 'end'])

In [45]:
exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
4527,chr1,6845383,6845635,23261,CAMTA1,NM_015215.3,1,False
4528,chr1,6880240,6880310,23261,CAMTA1,NM_015215.3,2,False
4529,chr1,6885151,6885270,23261,CAMTA1,NM_015215.3,3,False
4530,chr1,7151363,7151431,23261,CAMTA1,NM_015215.3,4,False
4531,chr1,7309550,7309686,23261,CAMTA1,NM_015215.3,5,False


In [46]:
exon.rename(columns={
    'chrom': '#chrom',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-exon-20200109.sorted.bed', index=False, sep='\t')

In [47]:
!bgzip -cf data/hi-exon-20200109.sorted.bed > ../data/hi-exon-20200109.sorted.bed.gz

In [48]:
!tabix -fp bed ../data/hi-exon-20200109.sorted.bed.gz

In [49]:
last_exon = exon[exon['last_exon']]

In [50]:
last_exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
4549,chr1,7826518,7829766,23261,CAMTA1,NM_015215.3,23,True
948,chr1,17345216,17345453,6390,SDHB,NM_003000.2,8,True
2185,chr1,27105513,27108601,8289,ARID1A,NM_139135.2,20,True
1875,chr1,27860755,27861427,27245,AHDC1,NM_001029882.3,7,True
872,chr1,43391045,43392912,6513,SLC2A1,NM_006516.2,10,True


In [51]:
last_exon_region = last_exon['chrom'] + ':' + last_exon['start'].astype(str) + '-' + last_exon['end'].astype(str)
last_exon_region = last_exon_region.str.replace('chr', '')

In [52]:
last_exon_region.head()

4549      1:7826518-7829766
948     1:17345216-17345453
2185    1:27105513-27108601
1875    1:27860755-27861427
872     1:43391045-43392912
dtype: object

In [53]:
need_fields = [
    'variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',
    'variants/AF_ESP', 'variants/AF_EXAC', 'variants/AF_TGP', 'variants/CLNSIG'
]

In [54]:
with open('data/clinvar_pathogenic_20191125.sorted.vcf', 'w') as f:
    headers = allel.read_vcf_headers('data/clinvar_20191125.vcf.gz')
    f.write(''.join(headers.headers))
    
    for region in last_exon_region:
        fields, samples, headers, it = allel.iter_vcf_chunks(
            'data/clinvar_20191125.vcf.gz', fields=need_fields, alt_number=1, region=region
        )
        for variants, *_ in it:
            esp_filter = np.isnan(variants['variants/AF_ESP'])
            esp_filter[~esp_filter] |= variants['variants/AF_ESP'][~esp_filter] < 0.01

            exac_filter = np.isnan(variants['variants/AF_EXAC'])
            exac_filter[~exac_filter] |= variants['variants/AF_EXAC'][~exac_filter] < 0.01

            tgp_filter = np.isnan(variants['variants/AF_TGP'])
            tgp_filter[~tgp_filter] |= variants['variants/AF_TGP'][~tgp_filter] < 0.01

            pathogenic_filter = np.isin(
                variants['variants/CLNSIG'], ['Likely_pathogenic', 'Pathogenic', 'Pathogenic/Likely_pathogenic']
            )

            af_filter = esp_filter & exac_filter & tgp_filter & pathogenic_filter

            filtered_variants = {k: v[af_filter] for k, v in variants.items()}

            filtered_variants['variants/CHROM'] = 'chr' + filtered_variants['variants/CHROM']

            names, callset = allel.normalize_callset(filtered_variants)
            allel.write_vcf_data(f, names, callset, None, {field: np.nan for field in need_fields})

In [55]:
!sed -i 's/AF_.\+=nan;//' data/clinvar_pathogenic_20191125.sorted.vcf

In [56]:
!bgzip -cf data/clinvar_pathogenic_20191125.sorted.vcf > ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [57]:
!tabix -fp vcf ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [58]:
uhi_genes = set(
    curation_gene.loc[curation_gene['Haploinsufficiency Score'] == '40', '#Gene Symbol']
)

In [59]:
uhi_gene = gene[gene['name2'].isin(uhi_genes)].copy()

In [60]:
uhi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
15408,chr1,55505148,55530526,255738,PCSK9,NM_174936.3,+
7326,chr13,20761601,20767114,2706,GJB2,NM_004004.5,-


In [61]:
uhi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uhi-gene-20200109.sorted.bed', index=False, sep='\t')

In [62]:
!bgzip -cf data/uhi-gene-20200109.sorted.bed > ../data/uhi-gene-20200109.sorted.bed.gz

In [63]:
!tabix -fp bed ../data/uhi-gene-20200109.sorted.bed.gz

In [64]:
region = pd.read_csv('data/ClinGen_region_curation_list_GRCh37.tsv', sep='\t', skiprows=5, dtype=str)

In [65]:
region.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN


In [66]:
position = region['Genomic Location'].str.extract(r'(?P<chrom>chr\w+)\s*:\s*(?P<start>\d+)\s*-\s*(?P<end>\d+)')
position['start'] = position['start'].astype(int)
position['end'] = position['end'].astype(int)

In [67]:
position.head()

,chrom,start,end
0,chr15,75631787,75972909
1,chrX,53363456,53793054
2,chr13,23544582,24881361
3,chr16,21570113,21740423
4,chr15,72963715,75508312


In [68]:
region_pos = region.merge(position, how='left', left_index=True, right_index=True)

In [69]:
region_pos.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID,chrom,start,end
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN,chr15,75631787,75972909
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN,chrX,53363456,53793054
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN,chr13,23544582,24881361
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN,chr16,21570113,21740423
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN,chr15,72963715,75508312


In [70]:
hi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [71]:
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
43,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD)
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
17,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3)
51,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (LCR-3/4-flanked...
63,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region


In [72]:
hi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/hi-region-20200109.sorted.bed', sep='\t', index=False)

In [73]:
!bgzip -cf data/hi-region-20200109.sorted.bed > ../data/hi-region-20200109.sorted.bed.gz

In [74]:
!tabix -fp bed ../data/hi-region-20200109.sorted.bed.gz

In [75]:
uhi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [76]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster)


In [77]:
uhi_region['genes'] = uhi_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [78]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere,
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster),"PSG8,PSG1,PSG6,PSG7,PSG11,PSG2,PSG5,PSG4"


In [79]:
uhi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uhi-region-20200109.sorted.bed', sep='\t', index=False)

In [80]:
!bgzip -cf data/uhi-region-20200109.sorted.bed > ../data/uhi-region-20200109.sorted.bed.gz

In [81]:
!tabix -fp bed ../data/uhi-region-20200109.sorted.bed.gz

In [82]:
with open('data/gene-table.json') as f:
    data = json.load(f)
    decipher = pd.DataFrame.from_records(data, columns=['name', 'hi_score', 'p_li']).rename(columns={'name': 'symbol'})
    decipher = decipher.merge(gene, left_on='symbol', right_on='name2')

In [83]:
decipher = decipher.loc[
    (decipher['p_li'] >= 0.9) & (decipher['hi_score'] < 0.1),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'p_li', 'hi_score']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [84]:
decipher.head()

,chrom,txStart,txEnd,GeneID,name2,name,p_li,hi_score
2112,chr1,2439969,2458067,55229,PANK4,NM_018216.2,0.945333,0.098966
2587,chr1,155290639,155300909,23623,RUSC1,NM_001105203.1,0.949429,0.035285
2705,chr1,232533710,232651354,57568,SIPA1L2,NM_020808.4,0.999220,0.079950
1608,chr10,852853,977645,23185,LARP4B,NM_015155.2,0.999886,0.071477
199,chr10,24872537,25012597,57584,ARHGAP21,NM_020824.3,0.999947,0.073197


In [85]:
decipher.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/decipher-gene-20200109.sorted.bed', sep='\t', index=False)

In [86]:
!bgzip -cf data/decipher-gene-20200109.sorted.bed > ../data/decipher-gene-20200109.sorted.bed.gz

In [87]:
!tabix -fp bed ../data/decipher-gene-20200109.sorted.bed.gz

In [88]:
ts_genes = set(
    curation_gene.loc[
        curation_gene['Triplosensitivity Score'] == '3', '#Gene Symbol'
    ]
)

In [89]:
ts_gene = gene[gene['name2'].isin(ts_genes)].copy()

In [90]:
ts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
20275,chr5,126112314,126172712,4001,LMNB1,NM_005573.3,+
12737,chrX,103031433,103047548,5354,PLP1,NM_001128834.2,+
21287,chrX,153287263,153363188,4204,MECP2,NM_004992.3,-


In [91]:
ts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/ts-gene-20200109.sorted.bed', index=False, sep='\t')

In [92]:
!bgzip -cf data/ts-gene-20200109.sorted.bed > ../data/ts-gene-20200109.sorted.bed.gz

In [93]:
!tabix -fp bed ../data/ts-gene-20200109.sorted.bed.gz

In [94]:
uts_genes = set(
    curation_gene.loc[curation_gene['Triplosensitivity Score'] == '40', '#Gene Symbol']
)

In [95]:
uts_gene = gene[gene['name2'].isin(uts_genes)].copy()

In [96]:
uts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
3727,chrX,6451658,6453159,51481,VCX3A,NM_016379.3,-


In [97]:
uts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uts-gene-20200109.sorted.bed', index=False, sep='\t')

In [98]:
!bgzip -cf data/uts-gene-20200109.sorted.bed > ../data/uts-gene-20200109.sorted.bed.gz

In [99]:
!tabix -fp bed ../data/uts-gene-20200109.sorted.bed.gz

In [100]:
ts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [101]:
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
62,chr15,22832519,28379874,ISCA-37404,"15q11q13 recurrent (PWS/AS) region (BP1-BP3, C..."
23,chr15,23747996,28379874,ISCA-37478,"15q11q13 recurrent (PWS/AS) region (BP2-BP3, C..."
64,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (..."
47,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...


In [102]:
ts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/ts-region-20200109.sorted.bed', sep='\t', index=False)

In [103]:
!bgzip -cf data/ts-region-20200109.sorted.bed > ../data/ts-region-20200109.sorted.bed.gz

In [104]:
!tabix -fp bed ../data/ts-region-20200109.sorted.bed.gz

In [105]:
uts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [106]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...


In [107]:
uts_region['genes'] = uts_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [108]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...,"TUBGCP5,CYFIP1,NIPA2,NIPA1"
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...,


In [109]:
uts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uts-region-20200109.sorted.bed', sep='\t', index=False)

In [110]:
!bgzip -cf data/uts-region-20200109.sorted.bed > ../data/uts-region-20200109.sorted.bed.gz

In [111]:
!tabix -fp bed ../data/uts-region-20200109.sorted.bed.gz